In [1]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton
import os
import json
from susy_qm import calculate_Hamiltonian

In [50]:
potential = 'AHO'
cutoff = 16
shots = 2

In [51]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [52]:
min_3_ev = eigenvalues.argsort()[:3]
min_eigenvector = np.asarray(eigenvectors[:, min_3_ev[1]])

In [53]:
operator_pool = []
for i in range(num_qubits):
    #operator_pool.append(qml.Identity(i))
    operator_pool.append(qml.PauliX(i))
    operator_pool.append(qml.PauliY(i))
    operator_pool.append(qml.PauliZ(i))


In [54]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)
basis_state = [1,0,0,0,0]

@qml.qnode(dev)
def circuit(times, op_list, try_cz=False, cz_wires=None, trotter_steps=1):

    qml.BasisState(basis_state, wires=range(num_qubits))

    for i, op in enumerate(op_list):
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            qml.ApproxTimeEvolution(op, time=times[i], n=trotter_steps)

    if try_cz:
        qml.CZ(wires=cz_wires)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [55]:
def cost_function(times, op_list):

    times = pnp.tensor(times, requires_grad=True)
    energy = circuit(times, op_list)
    
    return energy

In [56]:
dev = qml.device("default.qubit", wires=2*num_qubits, shots=shots)

@qml.qnode(dev)
def swap_test(trial_op, op_list, op_params, prev_op_list, prev_op_params, basis_state):

    trotter_steps=1
    bs = (basis_state + basis_state)
    qml.BasisState(bs, wires=range(2*num_qubits))
   
    prev_param_index = 0
    for pop in prev_op_list:

        if type(pop) == qml.CZ:
            qml.CZ(wires=pop.wires)
        else:
            qml.ApproxTimeEvolution(pop, time=prev_op_params[prev_param_index], n=trotter_steps)
            prev_param_index += 1
    

    param_index  = 0
    for op in op_list:
        if type(op) == qml.CZ:
            w0 = op.wires[0] + num_qubits
            w1 = op.wires[1] + num_qubits
            wire_map = {op.wires[0]:w0, op.wires[1]:w1}
            op = op.map_wires(wire_map)
            qml.CZ(wires=op.wires)
        else:
            wire = op.wires[0] + num_qubits
            wire_map = {op.wires[0]: wire}
            op = op.map_wires(wire_map)
            qml.ApproxTimeEvolution(op, time=op_params[param_index], n=trotter_steps)  
            param_index += 1

    qml.ApproxTimeEvolution(trial_op, time=0.0, n=trotter_steps)

    qml.Barrier()
    for i in range(num_qubits):
        qml.CNOT(wires=[i, i+num_qubits])    
        qml.Hadamard(wires=i)   

    prob = qml.probs(wires=range(2*num_qubits))

    return prob


In [57]:
def overlap(trial_op, op_list, op_params, prev_op_list, prev_op_params, basis_state):

    probs = swap_test(trial_op, op_list, op_params, prev_op_list, prev_op_params, basis_state)

    ol = 0
    for idx, p in enumerate(probs):

        bitstring = format(idx, '0{}b'.format(2*num_qubits))

        counter_11 = 0
        for i in range(num_qubits):
            a = int(bitstring[i])
            b = int(bitstring[i+num_qubits])
            if (a == 1 and b == 1):
                counter_11 +=1

        ol += p*(-1)**counter_11

    return ol

In [58]:
@qml.qnode(dev)
def grad_circuit(times, trial_op, op_list, op_params, trotter_steps=1):

    qml.BasisState(basis_state, wires=range(num_qubits))
    param_index = 0
    for op in op_list:
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            qml.ApproxTimeEvolution(op, time=op_params[param_index], n=trotter_steps)
            param_index +=1

    qml.ApproxTimeEvolution(trial_op, time=times, n=trotter_steps)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [59]:
def compute_grad(trial_op, op_list, op_params):
    t = pnp.tensor(0.0, requires_grad=True)
    grad_fn = qml.grad(grad_circuit)
    grad = grad_fn(t, trial_op, op_list, op_params)
    return grad

In [60]:
def grad_plus_overlap(trial_op, op_list, op_params, prev_op_list, prev_op_params, beta=200.0):

    grad = compute_grad(trial_op, op_list, op_params)
    
    penalty = 0
    if len(prev_op_list) != 0:
        for prev_op, prev_param in zip(prev_op_list, prev_op_params):
            ol = overlap(trial_op, op_list, op_params, prev_op, prev_param, basis_state)
            penalty += (beta*ol)
            if penalty > 0:
                print(f"Applying penalty {penalty} to op {trial_op}")
    
    return abs(grad), penalty

In [62]:
num_levels = 2
num_steps = 4
prev_op_list = []
prev_op_params = []

#variables
max_iter = 300
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

for e_level in range(num_levels):

    print("######################################")
    print(f"Energy level: {e_level}")

    op_list = []
    op_params = []

    for i in range(num_steps):

        print(f"step: {i}")

        grad_list = []
        overlaps = []

        for trial_op in operator_pool:
            grad, ol = grad_plus_overlap(trial_op, op_list, op_params, prev_op_list, prev_op_params)
            grad_list.append(grad)
            overlaps.append(ol)

        norm_grad = [i/sum(grad_list) for i in grad_list]
        results = [a + b for a, b in zip(norm_grad, overlaps)]
        maxidx = np.argmax(grad_list)
        op_list.append(operator_pool[maxidx])

        print(f"Best op is: {operator_pool[maxidx]}")

        # Generate Halton sequence
        num_dimensions = len(op_list)
        num_samples = popsize
        halton_sampler = Halton(d=num_dimensions)
        halton_samples = halton_sampler.random(n=num_samples)
        scaled_samples = 2 * np.pi * halton_samples

        bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]
        
        res = differential_evolution(cost_function,
                                        bounds,
                                        args=(op_list,),
                                        maxiter=max_iter,
                                        tol=tol,
                                        atol=abs_tol,
                                        strategy=strategy,
                                        popsize=popsize,
                                        init=scaled_samples,
                                        )
        
        if i!=0: 
            pre_min_e = min_e
            pre_op_params = op_params

        min_e = res.fun
        op_params = res.x

        print(f"Min E: {min_e}")
        print(res.success)

        if i!=0:
            if abs(pre_min_e - min_e) < 1e-8:
                print("gradient converged")
                op_list.pop()
                op_params = pre_op_params
                print(op_list, op_params)
                prev_op_list.append(op_list.copy())
                prev_op_params.append(op_params.copy())
                break


    

######################################
Energy level: 0
step: 0
Best op is: Y(1)
Min E: -0.0011669756806108533
True
step: 1
Best op is: X(1)
Min E: -0.0011669756806108533
True
gradient converged
[Y(1)] [2.97207465]
######################################
Energy level: 1
step: 0
Applying penalty 200.0 to op X(0)
Applying penalty 200.0 to op Y(0)
Applying penalty 200.0 to op Z(0)
Applying penalty 200.0 to op X(1)
Applying penalty 200.0 to op Z(1)
Applying penalty 200.0 to op X(2)
Applying penalty 200.0 to op Y(2)
Applying penalty 200.0 to op Z(2)
Applying penalty 200.0 to op X(3)
Applying penalty 200.0 to op Y(3)
Applying penalty 200.0 to op Z(3)
Applying penalty 200.0 to op X(4)
Applying penalty 200.0 to op Y(4)
Applying penalty 200.0 to op Z(4)
Best op is: X(2)
Min E: -0.0011669756806108533
True
step: 1
Applying penalty 200.0 to op X(0)
Applying penalty 200.0 to op Y(0)
Applying penalty 200.0 to op X(1)
Applying penalty 200.0 to op Y(1)
Applying penalty 200.0 to op Z(1)
Applying penalty 

In [45]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def final_circuit(params):

    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            pauli_string = qml.pauli.pauli_word_to_string(op)
            qml.PauliRot(params[params_index], pauli_string, wires=op.wires)
            params_index += 1

    return qml.state()

In [46]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_list))
print(qml.draw(final_circuit)(x0))

0: ─╭|Ψ⟩───────────┤  State
1: ─├|Ψ⟩──RY(5.64)─┤  State
2: ─├|Ψ⟩───────────┤  State
3: ─╰|Ψ⟩───────────┤  State


In [47]:
def overlap_function(params):

    params = pnp.tensor(params, requires_grad=True)
    ansatz_state = final_circuit(params)
    
    overlap = np.vdot(min_eigenvector, ansatz_state)
    cost = np.abs(overlap)**2  

    return (1 - cost)

In [48]:
bounds = [(0, 2 * np.pi) for _ in range(len(op_list))]

# Generate Halton sequence
num_dimensions = len(op_list)
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

#variables
max_iter = 300
strategy = "randtobest1bin"
tol = 1e-6
abs_tol = 1e-5
popsize = 20

print("Running for overlap")
overlap_res = differential_evolution(overlap_function,
                                    bounds,
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy,
                                    popsize=popsize,
                                    init=scaled_samples,
                                    )

Running for overlap


In [49]:
overlap_res.fun

np.float64(1.0)

In [32]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def energy_circuit(params):

    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            pauli_string = qml.pauli.pauli_word_to_string(op)
            qml.PauliRot(params[params_index], pauli_string, wires=op.wires)
            params_index += 1

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [36]:
x0 = overlap_res.x
#x0
energy_circuit(x0)

np.float64(51.48956061469692)

In [ ]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "DE",
        "num steps": num_steps,
        "basis_state": basis_state,
        "op_list": [str(o) for o in op_list],
        "overlap": overlap_res.fun,
        "hellinger": hf_res.fun
        }

In [ ]:
data

In [ ]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\TimeEv\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [ ]:
#variables
max_iter = 10000
strategy = "randtobest1bin"
tol = 1e-3
atol = 1e-3
popsize = 20

num_steps = 5
op_list = []
op_params = []

for i in range(num_steps):

    print(f"step: {i}")

    grad_list = []

    for op in operator_pool:
        grad = compute_grad(op, op_list, op_params)
        grad_list.append(abs(grad))

    maxidx = np.argmax(grad_list)
    op_list.append(operator_pool[maxidx])

    bounds = [(0, 2 * np.pi) for _ in range(len(op_list))]
    res = differential_evolution(cost_function,
                                    bounds,
                                    args=(op_list,),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=atol,
                                    strategy=strategy,
                                    popsize=popsize
                                    )
    if i!=0: pre_min_e = min_e
    min_e = res.fun
    pre_op_params = op_params
    op_params = res.x

    print(f"Min E: {min_e}")
    print(res.success)

    print("Testing CZ pool")
    cz_e = []
    for term in cz_pool:
        energy = circuit(op_params, op_list, try_cz=True, cz_wires=term.wires)
        cz_e.append(energy)

    min_cz_e = cz_e[np.argmin(cz_e)]
    min_cz_term = cz_pool[np.argmin(cz_e)]
    if min_cz_e < min_e:
        print(f"Adding {min_cz_term} reduces energy further")
        op_list.append(min_cz_term)
        min_e = min_cz_e
        print(f"Min E: {min_e}")
    
    if i!=0:
        if abs(pre_min_e - min_e) < 1e-8:
            print("gradient converged")
            op_list.pop()
            op_params = pre_op_params
            break


    